# Iris Classifier 

Basic Multilayer Perceptron (MLP) for the classic Iris problem using Keras.  Ideas and code originally from https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/.

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [ ]:
seed = 7
np.random.seed(seed)

### Load Dataset

Load the Iris data set from file.  Assumes file is in the same directory as this Notebook and that the file has no header.  The file can be obtained from the UCI Machine Learning Repository.  After loading it, the first 4 columns are selected into a features frame called X.  The label column (the 5th column) is selected into its own structure named Y.

In [ ]:
# load dataset
df = pd.read_csv("iris.data", header=None)
dataset = df.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]

### Prepare Dataset

The three feature columns are already prepped for learning since they're numeric.  We may consider some adjustment (such as standardizing them), but that's about improving accuracy as opposed to a compulsory adjustment.  The labels are categorical, though, meaning they are strings that specify the type of iris (three different types).  Much like other learning algorithms, we must encode these as numbers. Specific to MLP, we cannot simply number them ordinally (e.g. setosa = 1, virginica = 2, etc).  Instead, we must apply one-hot encoding so that 3 distinct labels result in 3 different columns.  We're used to doing this for feature columns and the practice here is otherwisde identical.  We can use scikit's LabelEncoder to produce numeric labels for them, then use Numpy's to_categorical function to produce the one-hot encoding from those integer labels. 

In [ ]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)


### Create Basic Model

Keras contains a number of high-level pieces to compose an MLP.  It does support almost infinite flexibility for advanced practitioners but in this case, we can make use of a couple basic ones to start.  

The essence of Keras (just like TensorFlow and CNTK in general) is to prepare a symbolic computation graph.  Envision this as a directed acylic graph that starts with a layer of input nodes, progresses through some number of hidden nodes (potentially several layers of them), and then terminates in an output node(s).  The exact number of nodes is the subject of much debate, scrutiny, and analysis: ultimately for any problem, you must spend time training models, evaluating their loss and error, and applying both empirically derived heuristics and some trial and error, refine it accordingly.  

The initial piece we'll use is a layer called Sequential.  By itself, it does nothing.  Its power is in providing the means for us to easily connect other layers together in a linear path, offloading much of the difficulty of reasoning about the data shapes and parameters into it.  In our case, we're going to build an MLP with a single hidden layer.  We have 4 input nodes (since there are 4 features).  We're going to start with 8 hidden nodes using a Keras layer called Dense.  We specify the input dimension to it since the input to the Sequential layer will come here first.  Remember that in an MLP, every input node is connected to every hidden node so there are 4*8 = 32 connections there.  Since each has a bias associated with it, that results in 40 parameters to be learned for juts this one layer.  The aspect of neural networks that gives them power is their choice of activation function.  Each node takes inputs and applies a basic dot product to the combination of internal weights and inputs to produce an output.  Instead of using the value directly, we achieve a non-linearity by using an activation function to convert the node's computed value into an actual output value (or activated value).  There are 3 activation functions in widespread use (though custom activation functions can be used as long as they adhere to certain criteria): sigmoid, hyperbolic tangent, and rectified linear.  If sigmoid looks familiar, this function is used in logistic regression.  In fact, you can realize that the difference between an MLP and logistic regression is the latter uses basically a single hidden node with sigmoid activation while our MLP is going to use 8 such nodes.  In practice, use of sigmoid has gone away in favor of tanh and predominately, rectified linear (ReLU).  ReLU is computed as f(x) = max(0,x), meaning it forces the output to be 0 or positive.  
We next need to connect our hidden layer to our output layer.  We again use a Dense layer for the output.  Yes, the same Dense layer as before.  From Keras' perspective, the layers are not different: it comes down to our usage.  We're doing 3-class classification so we need 3 output nodes.  But what do we want our output to be?  While we can set it up to output a binary class label, it tends to be more powerful to output probabilities for each class and that's what we'll do.  For that reason, we need 3 output nodes (1 per label) and the activated value of each will represent the probability that the instance in question is that label.  Softmax is the activation to achieve this.  

After building up the model, we have to compile it.  Before now, we were simply expressing our neural network as a graph with particular connections and interactions.  Now is when we actually determine the specifics for how we will train.  The objective of any learner is to minimize loss, defined as the difference between the current performance of the model and its objective function.  For this problem (multi-class classification), we use categorical cross-entropy.  If you were doing binary classification, you'd use "normal" cross-entropy.  There are several different optimizers you can choose from, the big 3 being stochastic gradient descent (SGD), Adam, and RMSProp (but other choices are AdaGrad, AdaDelta, and others).  By passing the text name to compile, we're instructing Keras to use the default values for the optimizer.  If we want to adjust the optimizer beyond its defaults, we can instantiate our own and then pass it in as the optimizer instead.  Lastly, we tell compile what metrics we're interested in seeing during training.  They are for our purposes in analyzing the learning - they are not used by the learning algorithm at all (which uses loss).  We'll keep track of accuracy.  

Having done all of this, we are almost ready to actually train the model.  What's missing?  The data! We've told it roughly what shape of data to expect (the input) and we've outlined how we want the connections to exist and the network architecture, but we have to now translate our data into whatever that form it is and start the learning. 

In [ ]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=4, activation='relu'))    
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

### Model Training

Choosing our training data is of critical importance to the accuracy of the resulting model.  We now have to figure out how to apportion training, validation, and test data among whatever data we have at hand.  In the Iris data set, everything we have access to is in a single file.  Like all machine learning, we must not show any test instances to the model during the learning process.  In addition, we may wish to hold back some training examples for validation purposes.  The main point is: all the same options normally available to you in data selection are available to you now.  For our case, we'll use the scikit-learn train_test_split to just split them into 2 (randomly selected) sets.  Other choices are k-fold cross validation, Stratified K-Fold, and of course home-grown solutions.

Calling fit on our model is what actually executes the training process. The minimum we have to do is provide the X and y (features and labels) as input.  However, generally we'll also specify a number of epochs, to what extent we want to use validation data (and how), and possibly configure callbacks for reporting.  An epoch is the number of times we want the learner to pass completely through the dataset for learning purposes.  1 epoch of course means the learner will see each instance 1 time, 2 epochs is 2 times, etc.  It's quite common to use a low number of epochs to establish that the model is properly configured and learning is occurring and use a higher number of epochs (in conjunction with early stopping) for live training.  In our case, we'll just use 20 epochs as an arbitrary number.  

In [ ]:
from sklearn.model_selection import train_test_split

(x_train, x_test, y_train, y_test) = train_test_split(X, dummy_y, test_size = .2, random_state = 4)

model = baseline_model()

model.fit(x_train, y_train, epochs = 50, validation_split=.2)



In [ ]:
# If you want to see more specifics about the model, use its summary function

model.summary()

In [ ]:
# Metrics names gets us the name of each metric the model can evaluate and 
# evaluate will do so against whatever test we provide (a test set, normally)
print (model.metrics_names)

model.evaluate(x_test, y_test)